In [77]:
import pandas as pd
import numpy as np

# data can be found in DEA

In [78]:
# find the corresponding index which contains str(value) in df
def locate_index(df, value):
    for i in df.index.astype(str):
        if value in i:
            return i

In [79]:
# interpolate cost for a given technology k from DEA database sheet
def cal_cost(sheet_name,k,usecols='B:F'):
    
    excel = pd.read_excel('inputs/technology_data_for_el_and_dh.xlsx',sheet_name=sheet_name,index_col=0,usecols=usecols)
    
    index = locate_index(excel, 'Financial data')
    index_loc_start = excel.index.get_loc(index)
    
    try:        
        index = locate_index(excel, 'Variable O&M')
        index_loc_end = excel.index.get_loc(index)
    
    except KeyError:        
        index = locate_index(excel, 'Fixed O&M')
        index_loc_end = excel.index.get_loc(index)

    df_raw = excel.iloc[index_loc_start+1:index_loc_end+1,:]
    
    df_raw = df_raw.append(excel.loc['Technical lifetime (years)'])

    if k in ['OCGT','CCGT','oil','biomass','central resistive heater']:
        index = locate_index(excel, 'efficiency')
        df_raw.loc['efficiency'] = excel.loc[index]/100
    elif 'heat pump' in k:
        df_raw.loc['efficiency'] = 4
    else:
        df_raw.loc['efficiency'] = 1
    
    df_raw.columns = [2015,2020,2030,2050]
    
    df_raw = df_raw.groupby(df_raw.index).sum(axis=0)

    df = pd.DataFrame(index=df_raw.index,columns=years)

    for index in df.index:
        
        values = np.interp(x=years,xp=df_raw.columns.values,fp=df_raw.loc[index,:].values.astype(float))
        
        df.loc[index,:] = values
        
    return df

In [80]:
sheet_names = {'onwind': '20 Onshore turbines',
               'offwind': '21 Offshore turbines',
               'OCGT': '52 OCGT - Natural gas',
               'CCGT': '05 Gas turb. CC, Back-pressure',
               'biomass CHP': '09 Straw, Large',
               'biomass HOP': '09 Straw HOP',
               'central coal CHP': '01 Coal CHP',
               'central heat pump': '40 Comp. heat pump, DH',
               'central resistive heater': '41 Electric Boilers',
               'central gas boiler': '44 Natural Gas DH Only'
              }

usecols = {}

In [81]:
years = np.arange(2020,2055,5)

d_by_tech = {}

# calculate costs and save results as a dict of df, indexed by cost, columned by year
for k in sheet_names.keys():
    
    sheet_name = sheet_names[k]
    
    print(k+' in PyPSA corresponds to '+sheet_name+' in DEA database.')
    
    df = cal_cost(sheet_name=sheet_name,k=k)
    
    df.fillna(value=0,inplace=True)
        
    d_by_tech[k] = df

onwind in PyPSA corresponds to 20 Onshore turbines in DEA database.
offwind in PyPSA corresponds to 21 Offshore turbines in DEA database.
OCGT in PyPSA corresponds to 52 OCGT - Natural gas in DEA database.
CCGT in PyPSA corresponds to 05 Gas turb. CC, Back-pressure in DEA database.
biomass CHP in PyPSA corresponds to 09 Straw, Large in DEA database.
biomass HOP in PyPSA corresponds to 09 Straw HOP in DEA database.
central coal CHP in PyPSA corresponds to 01 Coal CHP in DEA database.
central heat pump in PyPSA corresponds to 40 Comp. heat pump, DH in DEA database.
central resistive heater in PyPSA corresponds to 41 Electric Boilers in DEA database.
central gas boiler in PyPSA corresponds to 44 Natural Gas DH Only in DEA database.


In [82]:
# aggregate technologies into a dict, whose keys are years
d_by_year = {}

for year in years:
    
    index = ['investment','FOM','VOM','lifetime','efficiency']
    
    df = pd.DataFrame(index=index,columns=sheet_names.keys(),data=0,dtype=float)

    for tech in sheet_names.keys():

        index = locate_index(d_by_tech[tech], 'investment')

        CC = d_by_tech[tech].at[index,year]*1.e6 # in EUR/MW

        df.at['investment',tech] = np.round(CC/1.e3,0) # in EUR/KW

        index = locate_index(d_by_tech[tech], 'Fixed O&M')

        FOM = d_by_tech[tech].at[index,year] # in EUR/MW/year

        df.at['FOM',tech] = np.round(FOM/CC*100,3) # in %/year

        index = locate_index(d_by_tech[tech], 'Variable O&M')

        try:
            VOM = d_by_tech[tech].at[index,year] # in EUR/MWh
        except KeyError:
            VOM = 0

        df.at['VOM',tech] = VOM
        
        df.at['lifetime',tech] = d_by_tech[tech].at['Technical lifetime (years)',year]
        
        df.at['efficiency',tech] = d_by_tech[tech].at['efficiency',year]
        
    d_by_year[year] = df

In [83]:
# onwind, offwind, OCGT, CCGT, biomass electricity, central heat pump are from DEA
source_DEA = 'Technology Data for Energy Plants for Electricity and District heating generation'

# solar utility from Vartiaian 2019
data = np.interp(x=years,xp=[2020, 2030, 2040, 2050],fp=[431, 275, 204, 164])

solar_uti = pd.Series(data=data,index=years)

source_Vartiainen = 'Impact of weighted average cost of capital, capital expenditure, and other parameters on future utility‐scale PV levelised cost of electricity'

# solar rooftop from ETIP 2019
data = np.interp(x=years,xp=[2020, 2030, 2050],fp=[1150, 800, 550])

solar_roof = pd.Series(data=data,index=years)

source_ETIP = 'European PV Technology and Innovation Platform'

# nuclear, coal, lignite from Lazards
source_Lazards = 'Lazard’s Levelized Cost of Energy Analysis - Version 13.0'

In [84]:
# overwrite the cost assumption of 2030 for other years
for year in years:
    df = d_by_year[year]
    costs = pd.read_csv('inputs/costs_PyPSA.csv', index_col=list(range(2))).sort_index()
    
    # central heat pump for district heating from DEA
    costs.rename({'central air-sourced heat pump':'central heat pump'},inplace=True)
    costs.rename({'decentral air-sourced heat pump':'decentral heat pump'},inplace=True)
    costs.drop(index='decentral ground-sourced heat pump',inplace=True)
    
    # central CHP is gas-fired 
    costs.rename({'central CHP':'central gas CHP'},inplace=True)
    
    for tech in df.columns:
        for para in df.index:
            costs.at[(tech, para), 'value'] = df.at[para, tech]
            costs.at[(tech, para), 'source'] = source_DEA
            
    # solar utility from Vartiaian 2019
    costs.loc[('solar-utility','investment'),'value'] = solar_uti[year]
    costs.loc[('solar-utility','investment'),'source'] = source_Vartiainen
    
    costs.loc[('solar-utility','lifetime'),'value'] = 30
    costs.loc[('solar-utility','lifetime'),'source'] = source_Vartiainen
    
    # solar rooftop from ETIP 2019
    costs.loc[('solar-rooftop','investment'),'value'] = solar_roof[year]
    costs.loc[('solar-rooftop','investment'),'source'] = source_ETIP
    
    costs.loc[('solar-rooftop','lifetime'),'value'] = 30
    costs.loc[('solar-rooftop','lifetime'),'source'] = source_ETIP
    
    # nuclear from Lazards
    costs.loc[('nuclear','investment'),'value'] = 8595
    costs.loc[('nuclear','investment'),'source'] = source_Lazards
    
    costs.loc[('nuclear','FOM'),'value'] = 1.4
    costs.loc[('nuclear','FOM'),'source'] = source_Lazards
    
    costs.loc[('nuclear','VOM'),'value'] = 3.5
    costs.loc[('nuclear','VOM'),'source'] = source_Lazards
    
    costs.loc[('nuclear','efficiency'),'value'] = 0.33
    costs.loc[('nuclear','efficiency'),'source'] = source_Lazards
    
    costs.loc[('nuclear','fuel'),'value'] = 2.6
    costs.loc[('nuclear','fuel'),'source'] = source_Lazards
    costs.loc[('uranium','fuel'),'value'] = 2.6
    costs.loc[('uranium','fuel'),'source'] = source_Lazards
    
    costs.loc[('nuclear','lifetime'),'value'] = 40
    costs.loc[('nuclear','lifetime'),'source'] = source_Lazards
    
    # coal from Lazards and BP 2019
    costs.loc[('coal','investment'),'value'] = 4162.5
    costs.loc[('coal','investment'),'source'] = source_Lazards
    
    costs.loc[('coal','FOM'),'value'] = 1.6
    costs.loc[('coal','FOM'),'source'] = source_Lazards
    
    costs.loc[('coal','VOM'),'value'] = 3.5
    costs.loc[('coal','VOM'),'source'] = source_Lazards
    
    costs.loc[('coal','efficiency'),'value'] = 0.33
    costs.loc[('coal','efficiency'),'source'] = source_Lazards
    
    costs.loc[('coal','fuel'),'value'] = 8.15
    costs.loc[('coal','fuel'),'source'] = 'BP 2019'
    
    costs.loc[('coal','lifetime'),'value'] = 40
    costs.loc[('coal','lifetime'),'source'] = source_Lazards
    
    # lignite from Lazards and DIW
    costs.loc[('lignite','investment'),'value'] = 4162.5
    costs.loc[('lignite','investment'),'source'] = source_Lazards
    
    costs.loc[('lignite','FOM'),'value'] = 1.6
    costs.loc[('lignite','FOM'),'source'] = source_Lazards
    
    costs.loc[('lignite','VOM'),'value'] = 3.5
    costs.loc[('lignite','VOM'),'source'] = source_Lazards
    
    costs.loc[('lignite','efficiency'),'value'] = 0.33
    costs.loc[('lignite','efficiency'),'source'] = source_Lazards
    
    costs.loc[('lignite','fuel'),'value'] = 2.9
    costs.loc[('lignite','fuel'),'source'] = 'DIW'

    costs.loc[('lignite','lifetime'),'value'] = 40
    costs.loc[('lignite','lifetime'),'source'] = source_Lazards
      
    
    costs.fillna(0,inplace=True)
    costs.to_csv('outputs/costs_{}.csv'.format(year))